<a href="https://colab.research.google.com/github/monp97/dataset/blob/master/gender_classification_66.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! git clone https://github.com/monp97/dataset.git

Cloning into 'dataset'...
remote: Enumerating objects: 7691, done.
remote: Counting objects: 100% (7691/7691), done.
remote: Compressing objects: 100% (7431/7431), done.
remote: Total 70630 (delta 267), reused 7677 (delta 260), pack-reused 62939
Receiving objects: 100% (70630/70630), 454.06 MiB | 31.09 MiB/s, done.
Resolving deltas: 100% (3255/3255), done.
Checking out files: 100% (128230/128230), done.


In [0]:
import shutil
shutil.rmtree('/content/final_gender_dataset')

In [2]:
! pip install split-folders

In [11]:
import os

directory = "final_gender_dataset"
  
# Parent Directory path 
parent_dir = "/content/"
  
# Path 
path = os.path.join(parent_dir, directory) 
  
# Create the directory 
# 'GeeksForGeeks' in 
# '/home / User / Documents' 
os.mkdir(path) 
print("Directory '% s' created" % directory) 

Directory 'final_gender_dataset' created


In [0]:
directory1 = "training"
directory2 = "test"

# Parent Directory path 
parent_dir = "/content/final_gender_dataset"
  
# Path 
path1 = os.path.join(parent_dir, directory1) 
path2 = os.path.join(parent_dir, directory2) 
  
# Create the directory 
# 'GeeksForGeeks' in 
# '/home / User / Documents' 
os.mkdir(path1) 
os.mkdir(path2) 

In [0]:
directory1 = "female"
directory2 = "male"

# Parent Directory path 
parent_dir = "/content/final_gender_dataset/training"
  
# Path 
path1 = os.path.join(parent_dir,directory1) 
path2 = os.path.join(parent_dir, directory2) 
  
# Create the directory 
# 'GeeksForGeeks' in 
# '/home / User / Documents' 
os.mkdir(path1) 
os.mkdir(path2) 

directory1 = "female"
directory2 = "male"

# Parent Directory path 
parent_dir = "/content/final_gender_dataset/test"
  
# Path 
path1 = os.path.join(parent_dir, directory1) 
path2 = os.path.join(parent_dir, directory2) 
  
# Create the directory 
# 'GeeksForGeeks' in 
# '/home / User / Documents' 
os.mkdir(path1) 
os.mkdir(path2) 

In [0]:
import numpy as np
import cv2
for filename in os.listdir('/content/dataset/Real'):
        subject_id, etc = filename.split('__')
        gender, lr, finger, _ = etc.split('_')
        src='/content/dataset/Real/'+filename
        image = cv2.imread(src)

        proc_img=(image)

        # refer = {
        #   "index": "1",
        #   "middle": "2",
        #   "ring":"3",
        #   "little":"4",
        #   "thumb":"5"
        # }

        dir_name=''

        if gender=='M':
          dir_name=dir_name+'male'
        else:
          dir_name=dir_name+'female'
          

        # if lr=='Left':
        #   dir_name=dir_name+'left'
        # else:
        #   dir_name=dir_name+'right'

        # dir_name=dir_name+refer[finger]

        # name=dir_name

        # dir_name=name+'_'+str(dic[name])
        # final_filename=filename[:-4]+'.png'
        cv2.imwrite(os.path.join('/content/final_gender_dataset/training/'+dir_name+'/'+filename),proc_img)

In [17]:
import split_folders

# Split with a ratio.
# To only split into training and validation set, set a tuple to `ratio`, i.e, `(.8, .2)`.
split_folders.ratio('/content/final_gender_dataset/training', output="output", seed=1337, ratio=(.8, .1, .1)) # default values

Copying files: 2460 files [00:00, 5986.04 files/s]


In [0]:
import os
import random
# Go over each folder path
files = os.listdir('/content/final_gender_dataset/training/male')  # Get filenames in current folder

files = random.sample(files,3540)  # Pick 900 random files
for file in files:
  f = os.path.join('/content/final_gender_dataset/training/male', file)  # Create valid path to file
  os.remove(f)  

In [18]:
dir_female_1=os.listdir('/content/final_gender_dataset/training/female')
dir_male_1=os.listdir('/content/final_gender_dataset/training/male')
dir_female_2=os.listdir('/content/final_gender_dataset/test/female')
dir_male_2=os.listdir('/content/final_gender_dataset/test/male')

print(len(dir_female_1))
print(len(dir_male_1))
print(len(dir_female_2))
print(len(dir_male_2))


1230
1230
0
0


In [19]:
from keras.preprocessing.image import ImageDataGenerator
import numpy as np

train_datagen=ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True, 
    fill_mode='nearest'
)

test_datagen=ImageDataGenerator(rescale=1./255)


training_set=train_datagen.flow_from_directory(
    '/content/output/train',
    target_size=(96,96),
    batch_size=8,
    class_mode='binary',
    shuffle=True,
)


val_set=test_datagen.flow_from_directory(
    '/content/output/val',
    target_size=(96,96),
    batch_size=8,
    class_mode='binary',
    shuffle=False
)


Using TensorFlow backend.


Found 1968 images belonging to 2 classes.
Found 246 images belonging to 2 classes.


In [0]:
from keras import optimizers
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator
 
def define_model():
  model = Sequential()
  model.add(Conv2D(32,(3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(96, 96, 3)))
  model.add(MaxPooling2D((2, 2)))
  model.add(Flatten())
  model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
  model.add(Dense(1, activation='sigmoid'))
  opt = optimizers.adam(lr=0.0001)
  model.compile(optimizer=opt,loss='binary_crossentropy',metrics=['accuracy'])
  return model

In [27]:
from keras.applications.resnet50 import ResNet50, preprocess_input

base_model = ResNet50(weights='imagenet', 
                      include_top=False, 
                      input_shape=(96, 96, 3))

/usr/local/lib/python3.6/dist-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


94658560/94653016 [==============================] - 3s 0us/step


In [0]:
from keras.layers import Dense, Activation, Flatten, Dropout
from keras.models import Sequential, Model

def build_finetune_model(base_model, dropout, fc_layers, num_classes):
    for layer in base_model.layers:
        layer.trainable = False

    x = base_model.output
    x = Flatten()(x)
    for fc in fc_layers:
        # New FC layer, random init
        x = Dense(fc, activation='relu')(x) 
        x = Dropout(dropout)(x)

    # New softmax layer
    predictions = Dense(num_classes, activation='sigmoid')(x) 
    
    finetune_model = Model(inputs=base_model.input, outputs=predictions)

    return finetune_model

class_list = ["left", "right"]
FC_LAYERS = [1024, 1024]
dropout = 0.5

finetune_model = build_finetune_model(base_model, 
                                      dropout=dropout, 
                                      fc_layers=FC_LAYERS, 
                                      num_classes=1)

In [38]:
from IPython.display import display
from PIL import Image

# mod=define_model()
from keras import optimizers
opt = optimizers.adam(lr=0.0001)
finetune_model.compile(optimizer=opt,loss='binary_crossentropy',metrics=['accuracy'])
finetune_model.fit_generator(training_set,
                    steps_per_epoch=training_set.samples // 8,
                    epochs=100,
                    validation_data=val_set,
                    nb_val_samples= val_set.samples //8)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  if sys.path[0] == '':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras_pre..., steps_per_epoch=246, epochs=100, validation_data=<keras_pre..., validation_steps=30)`
  if sys.path[0] == '':


Epoch 1/100
246/246 [==============================] - 11s 44ms/step - loss: 0.5899 - acc: 0.6880 - val_loss: 0.7047 - val_acc: 0.4875
Epoch 2/100
246/246 [==============================] - 7s 30ms/step - loss: 0.5842 - acc: 0.7058 - val_loss: 0.7067 - val_acc: 0.4832
Epoch 3/100
246/246 [==============================] - 7s 29ms/step - loss: 0.5699 - acc: 0.7160 - val_loss: 0.7025 - val_acc: 0.4832
Epoch 4/100
246/246 [==============================] - 7s 30ms/step - loss: 0.5727 - acc: 0.7022 - val_loss: 0.7025 - val_acc: 0.4832
Epoch 5/100
246/246 [==============================] - 7s 29ms/step - loss: 0.5687 - acc: 0.7053 - val_loss: 0.7063 - val_acc: 0.4832
Epoch 6/100
246/246 [==============================] - 7s 29ms/step - loss: 0.5756 - acc: 0.7241 - val_loss: 0.7086 - val_acc: 0.4832
Epoch 7/100
246/246 [==============================] - 7s 30ms/step - loss: 0.5565 - acc: 0.7144 - val_loss: 0.7070 - val_acc: 0.4832
Epoch 8/100
246/246 [==============================] - 7s 29m

In [39]:
test_datagen=ImageDataGenerator(rescale=1./255)
test_set=train_datagen.flow_from_directory(
    '/content/output/test',
    target_size=(96,96),
    batch_size=8,
    class_mode='binary',
    shuffle=False
)

_, acc = mod.evaluate_generator(test_set, steps=len(test_set), verbose=0)
print('Test Accuracy: %.3f' % (acc * 100))

Found 246 images belonging to 2 classes.
Test Accuracy: 66.260


In [40]:
from sklearn.metrics import classification_report, confusion_matrix
Y_pred =mod.predict_generator(test_set, 246 //8)
# print(Y_pred[:20])

for i in range(240):
  if Y_pred[i]<=0.5:
    Y_pred[i]=0
  else:
    Y_pred[i]=1

# print(Y_pred)
y_pred = np.argmax(Y_pred, axis=1)
# print('hi')
# print(y_pred[:20])
print('Confusion Matrix')
print(confusion_matrix(test_set.classes[:240],Y_pred))

Confusion Matrix
[[76 47]
 [37 80]]
